In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

In [3]:
import sys
sys.path.insert(0, '../../toyaikit/')

In [4]:
import os

In [5]:
from openai import OpenAI

In [6]:
client = OpenAI(api_key=os.getenv('ZAI_API_KEY'), base_url='https://api.z.ai/api/paas/v4/')

In [7]:
def convert_single_tool(tool):
    """
    Convert a single OpenAI tool/function API dict to Chat Completions function format.
    """
    if tool["type"] != "function":
        raise "it's not a function"

    return {
        "type": "function",
        "function": {
            "name": tool["name"],
            "description": tool["description"],
            "parameters": tool["parameters"]
        }
    }

def convert_api_tools_to_chat_functions(api_tools):
    """
    Convert a list of OpenAI API tools to Chat Completions function format.
    """
    chat_functions = []

    for tool in api_tools:
        converted = convert_single_tool(tool)
        chat_functions.append(converted)

    return chat_functions

def convert_function_output_to_tool_message(data):
    return {
        "role": "tool",
        "tool_call_id": data["call_id"],
        "content": data["output"]
    }

In [8]:
project_path = 'todo-test'

In [47]:
from pathlib import Path
import tools

agent_tools = tools.AgentTools(Path(project_path))

In [48]:
from toyaikit.tools import Tools

In [49]:
tools_obj = Tools()
tools_obj.add_tools(agent_tools)
chat_functions = convert_api_tools_to_chat_functions(tools_obj.get_tools())

In [50]:
from toyaikit.chat import IPythonChatInterface

chat_interface = IPythonChatInterface()

In [51]:
class D(dict):
    def __getattr__(self, key):
        value = self.get(key)
        if isinstance(value, dict):
            return D(value)
        return value

In [54]:
developer_prompt = """
You are a coding agent. Your task is to modify the provided Django project template
according to user instructions. You don't tell the user what to do; you do it yourself using the 
available tools. First, think about the sequence of steps you will do, and then 
execute the sequence.
Always ensure changes are consistent with Django best practices and the project’s structure.

## Project Overview

The project is a Django 5.2.4 web application scaffolded with standard best practices. It uses:
- Python 3.8+
- Django 5.2.4 (as specified in pyproject.toml)
- uv for Python environment and dependency management
- SQLite as the default database (see settings.py)
- Standard Django apps and a custom app called myapp
- HTML templates for rendering views
- TailwindCSS for styling

## File Tree


├── .python-version
├── README.md
├── manage.py
├── pyproject.toml
├── uv.lock
├── myapp/
│   ├── __init__.py
│   ├── admin.py
│   ├── apps.py
│   ├── migrations/
│   │   └── __init__.py
│   ├── models.py
│   ├── templates/
│   │   └── home.html
│   ├── tests.py
│   └── views.py
├── myproject/
│   ├── __init__.py
│   ├── asgi.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── templates/
    └── base.html

## Content Description

- manage.py: Standard Django management script for running commands.
- README.md: Setup and run instructions, including use of uv for dependency management.
- pyproject.toml: Project metadata and dependencies (Django 5.2.4).
- uv.lock: Lock file for reproducible Python environments.
- .python-version: Specifies the Python version for the project.
- myapp/: Custom Django app with models, views, admin, tests, and a template (home.html).
  - migrations/: Contains migration files for database schema.
- myproject/: Django project configuration (settings, URLs, WSGI/ASGI entrypoints).
  - settings.py: Configures installed apps, middleware, database (SQLite), templates, etc.
- templates/: Project-level templates, including base.html.

You have full access to modify, add, or remove files and code within this structure using your available tools.


## Additional instructions

- Don't execute "runproject", but you can execute other commands to check if the project is working.
- Make sure you use Tailwind styles for making the result look beatiful
- Use pictograms and emojis when possible. Font-awesome is awailable
- Avoid putting complex logic to templates - do it on the server side when possible
- Make multiple function calls at once when needed
"""

In [ ]:
chat_messages = [
    {"role": "system", "content": developer_prompt},
]    

while True:
    user_input = chat_interface.input()
    if user_input.lower() == 'stop':
        chat_interface.display('Chat ended')
        break

    chat_messages.append({"role": "user", "content": user_input})

    while True:
        reponse = client.chat.completions.create(
            model="glm-4.5",
            messages=chat_messages,
            tools=chat_functions
        )

        first_choice = reponse.choices[0]
        message_response = first_choice.message
        chat_messages.append(message_response)

        if hasattr(message_response, 'reasoning_content'):
            reasoning = message_response.reasoning_content.strip()
            if reasoning != "":
                chat_interface.display_reasoning(reasoning)

        content = message_response.content.strip()
        if content != "":
            chat_interface.display_response(content)

        if hasattr(message_response, 'tool_calls'):
            calls = message_response.tool_calls
        else:
            calls = []
        
        if len(calls) == 0:
            break
        
        for call in calls:
            function_call = D(call.function.model_dump())
            function_call['call_id'] = call.id

            call_result = tools_obj.function_call(function_call)
            call_result = convert_function_output_to_tool_message(call_result)

            chat_messages.append(call_result)

            chat_interface.display_function_call(
                function_name=function_call.name,
                arguments=function_call['arguments'],
                result=call_result['content']
            )

You: todo
